# Tello drone flight with simultaneous video capture

# Introduction

The strategy is described in this [video tutorial](https://www.youtube.com/watch?v=HYJ3vcHG4jY&list=PLAu6u02VrrtzcivPSOqV5EHZWqmUyEX2o&index=10) by Jacob Pitsenberger with code from [this github repo](https://github.com/Jacob-Pitsenberger/Tello-Flight-Routine-with-Video-Stream)

This code uses threading to execute a flight routine with simultaneous video streaming. 
* Two functions run at the same time: call video_streaming() as a target thread, implement it as a daemon so it runs in the background.
* Control timing of function execution: define an event object: flight_routine() waits for variable in video_streaming() to indicate it is ready


# Setup

In [1]:
import threading # threading to have to functions execute simultaneously
from djitellopy import tello  # djitellopy: drone interaction: tello class to define drone object
import cv2  # Import OpenCV for video frame operations

# Functions

Function to define flight routine

In [2]:
def flight_routine(my_drone):
    print("waiting for event to signal video stream readiness")
    stream_ready.wait()

    print("event signaled for video stream readiness")

    # Send the takeoff command, movement commands, and lastly, the land command
    print("takeoff\n")
    my_drone.takeoff()
    print("move forward\n")
    my_drone.move_forward(20) # move forward 20 cm
    print("move up\n")
    my_drone.move_up(20)
    print("move back\n")
    my_drone.move_back(20)
    print("move down\n")
    my_drone.move_down(20)
    print("rotate CW\n")
    my_drone.rotate_clockwise(180)
    print("rotate CCW\n")
    my_drone.rotate_counter_clockwise(180)
    print("land")
    my_drone.land()

Function to get sensory data (battery level)

In [3]:
def get_status_data(my_drone):
    """Acquire other status data and return as dictionary"""
    status_data = {
        'flight_time':my_drone.get_flight_time(),
        'battery':my_drone.get_battery(),
        'height':my_drone.get_height(),
        'tof':my_drone.get_distance_tof(),
        'temph':my_drone.get_highest_temperature(),
        'barometer':my_drone.get_barometer()
    }
    return(status_data)

Function to control video streaming

In [4]:
def stream_video(my_drone):
    while True:
    # get sensor data 
        my_status_data = get_status_data(my_drone)

        frame = my_drone.get_frame_read().frame

        # print sensor data to screen
        cv2.putText(frame, "Battery " + str(my_status_data['battery']) + " percent", (10, 20), 
                cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 234, 255), 1,2)
        
        cv2.imshow('tello stream', frame)

        # Check if streaming ready. 
        if not stream_ready.is_set():

            # Signal that video streaming is ready
            stream_ready.set()
            print("Event Signal Set: Stream is live.")

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
    cv2.destroyAllWindows()

# Main program

Execute flight routine with video. type 'q' to quit

In [ ]:
# create event object from threading module (object = module.function())
stream_ready = threading.Event()

# Initialize my_drone as tello object, connect to it, and turn its video stream on.
my_drone = tello.Tello() # object = instance.Class()
my_drone.connect()
my_drone.streamon()
print("drone connected and stream on. Starting video stream thread.\n")

# set stream_video as thread target
stream_thread = threading.Thread(target=stream_video, args=(my_drone,))

# Set thread as a daemon and start it
stream_thread.daemon = True
stream_thread.start()

# Execute the qflight routine
flight_routine(my_drone)

print("Flight routine ended. Rebooting drone now...")

# Reboot the drone at the end
drone.reboot()